In [1]:
import pandas as pd
import numpy as np 
import sqlite3
import re

In [2]:
from bengali_stemmer.rafikamal2014 import RafiStemmer
stemmer = RafiStemmer()

In [3]:
train_data = pd.read_csv('ulm_train.csv')
stopwords = pd.read_csv('Stopwords.csv')
test_data = pd.read_csv('ulm_test.csv')

In [4]:
train_data

,label,text,is_valid
0,fe,সাম্প্রদায়িক সংঘাত ও মানবিক প্রতিরোধ সাম্প্রদা...,False
1,ij,আজ মাও সেতুং-এর জন্মদিন। মাও সে তুং। বারবার ইত...,False
2,mk,"গোলাম আযমের বিরুদ্ধে অভিযোগ ৬১ টি, যেকোনো ১ টি...",False
3,rn,জলের সন্তরন- (এক) অনেক গুলো সাপ মিজানের শরীরের...,False
4,hm,খুবরগুড়ে . . . এইভাবে হতে থাকে ক্রমাগত কেউ মার...,False
...,...,...,...
1795,rn,রহস্যময় বিতর্কিত পুরুষ রাসপুতিন ইতিহাস কি শুধু...,False
1796,fe,রাজনৈতিক মূর্খতা ও মুক্তিযুদ্ধের মৌলিক ফসল রাজ...,False
1797,mk,ইউপি ইলেকশন এবং ___ বাংলাদেশের তিন স্তরবিশিষ্ট...,False
1798,rn,"মনে করি, A হলো একটি সফল জীবন। A=x+y+z, এখানে x...",False


In [5]:
def to_encode(bangla_word):
    return bangla_word.encode('utf-8')


def to_decode(bangla_word):
    return bangla_word.decode('utf-8')

In [6]:
set_stop = set(stopwords['words'].map(to_encode)) #to encoded stop words

In [7]:
texts = train_data['text'].values

In [8]:
def clean_punct(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#|।|’|‘]', r'', sentence) #when find this types of expression then substitute to space
    cleaned1 = re.sub(r'[.|,|(|)|\|/]', r'', cleaned)
    cleaned = re.sub(r'[০|১|২|৩|৪|৫|৬|৭|৮|৯]', r'', cleaned1)
    cleaned1 = re.sub(r'[-|=]', r' ', cleaned)
    return cleaned1

In [9]:
def pre_process(data):
    i=0
    str1=' '
    final_string = []
    final_words = []
    all_negative_words = []
    s=''

    for sentence in data:
        filtered_sentence = []

        for w in sentence.split():
            for cleaned_word in clean_punct(w).split():
                if len(cleaned_word)>2:
                    if(to_encode(cleaned_word) not in set_stop):
                        s = stemmer.stem_word(cleaned_word)
                        if len(s)>2:
                            final_words.append(s)
                            filtered_sentence.append(s)
                    else:
                        continue
                else:
                    continue

        str1 = " ".join(filtered_sentence) #the str1 added each line and finally add into final_string
        final_string.append(str1)
    return final_string

In [10]:
train_data['text'].values[1]

'আজ মাও সেতুং-এর জন্মদিন। মাও সে তুং। বারবার ইতিহাসের চাকা বদলে দিয়েছেন চৈনিক এই মানুষটি। মার্কসের বিরুদ্ধে দাঁড়িয়ে শ্রমিকের বদলে কৃষককে চিহ্নিত করেছেন বিপ্লবের মূল চালিকা শক্তি হিসেবে, গড়ে তুলেছেন সশস্ত্র রেড আর্মি, প্রচলন করেছেন আরণ্যক গেরিলা যুদ্ধের। সত্য এই- বিশ্বময় তরুণেরা আজও হাঁটছে তাঁর দেখানো পথে। আজও নেপাল থেকে লাতিন আমেরিকার বিপ্লবী তরুণেরা পার্বত্য আরণ্যক এলাকায় নিঘূর্ম রাত কাটাচ্ছে চেয়ারম্যান মাও-এর নির্দেশে। চিনে হুনান নামে একটি প্রদেশ রয়েছে। সেই হুনান প্রদেশেই রয়েছে শাওশান গ্রাম। ১৮৯৩। ২৬ ডিসেম্বর। মাও সে তুং-এর জন্ম সেই শাওশান গ্রামেই। পরিবারও ছিল স্বচ্ছল। ভালো স্কুলে পড়েছেন। বাবা ছিলেন কনফুসিয়াসপন্থি। মা ছিলেন একনিষ্ট বৌদ্ধ। কাজেই ছেলেবেলায় এ দুটি মতে বিশ্বাসী হয়ে উঠেছিলেন। হুনান প্রদেশের রাজধানীর নাম চাঙশা। ১৯১১। মাও চাঙশা-য় চলে এলেন পড়তে। ভর্তি হলেন হুনান টির্চাস কলেজে । এখানেই প্রথম পাশ্চাত্য দর্শন সম্বন্ধে জানতে পারেন। তা হলে বুদ্ধ ও কনফুসিয়াস সব কথা বলে যাননি? যাক। সে সময়টায় চিনে চলছিল কিঙ রাজতন্ত্রের দুঃশাসন। তার বিরুদ্ধে জাতীয়তাবাদীদের তীব্র গনআন্দোলন হচ্ছিল। সান

In [11]:
texts

array(['সাম্প্রদায়িক সংঘাত ও মানবিক প্রতিরোধ সাম্প্রদায়িক সংঘাত ও মানবিক প্রতিরোধফকির ইলিয়াস=======================================আমি যে দেশটিতে থাকি এখানে বহুজাতিক, বহুভাষিক মানুষের বাস। অনেক ধর্মাবলম্বী, মতাবলম্বী মানুষ। কারো সঙ্গে কারো কোনো মিল নেই। এরা চাইলে কিন্তু খুব সামান্য বিষয় নিয়েই প্রতিদিন দাঙ্গা করতে পারতো। না- তেমনটি এখানে হচ্ছে না। হ্যাঁ, প্রিয় পাঠক- আমি মার্কিন যুক্তরাষ্ট্রের কথা বলছি।এটি এমন একটি দেশ, মানুষ মানুষের বুকের পাঁজর চিবিয়ে খেয়ে ফেলতো! যদি এ দেশে কঠোর আইন না থাকতো। না- তারা তা পারছে না। পারবে না। পারবে না এ জন্য, এমন কিছু করলে তাদের কঠিন শাস্তির মুখোমুখি হতে হবে। তাই এ জীবনবাজি রেখে ‘রায়ট’ করবে কে?পাক-ভারত উপমহাদেশ জন্ম নিয়েছে কিংবা বিভক্ত হয়েছে ‘রায়ট’-এর মধ্য দিয়ে। কী ঘটেছিল- তা আমাদের কারোরই অজানা নয়। এ জন্মইতিহাস নিয়েই জন্মেছে পাকিস্তান-ভারত, পরবর্তীকালে বাংলাদেশ। একটি ভূখণ্ডে ‘দ্বিজাতি তত্ত্ব’, ‘শত্রু সম্পত্তি আইন’ করা হয়েছে। মানুষে মানুষে ধর্মের বিভাজন করা হয়েছে সেভাবেই খুব পরিকল্পিতভাবে।১৯৪৭ সালে ভারত ভাগের সময় অসংখ্য হিন্দু তাঁদের ঘরবাড়ি ছেড়ে চলে গিয়েছ

In [12]:
train_data["text"].values[2]

"গোলাম আযমের বিরুদ্ধে অভিযোগ ৬১ টি, যেকোনো ১ টি প্রমাণিত হলেই ফাঁসি !!! গোলাম আযমের বিরুদ্ধে ৫ ধরনের ঘটনায় ৬১টি অভিযোগ আমলে নেয় আন্তর্জাতিক অপরাধ ট্রাইব্যুনাল।একাত্তরে মানবতাবিরোধী অপরাধে জামায়াতের সাবেক আমির গোলাম আযমের বিরুদ্ধে ৫ ধরনের ঘটনায় ৬১টি অভিযোগ আমলে নেয় আন্তর্জাতিক অপরাধ ট্রাইব্যুনাল।৬১ অভিযোগ তার বিরুদ্ধে, পাকিস্তানি বাহিনীকে সহায়তা ও তাদের সঙ্গে চক্রান্তের দায়ে ৬টি, পরিকল্পনার দায়ে ৩টি, উস্কানির দায়ে ২৮টি, সম্পৃক্ততার দায়ে ২৪টি এবং ব্যক্তিগতভাবে হত্যা ও নির্যাতনের দায়ে ১টিসহ মোট ৬২টি অভিযোগ আনা হয়। এর মধ্যে একটি অভিযোগ বাদ দেন ট্রাইব্যুনাল। একাত্তরের ২১ নভেম্বর গভীর রাতে ব্রাহ্মণবাড়িয়া কারাগারের ৩৮ জনকে শহরের পৈরতলা রেলব্রিজের কাছে নিয়ে গুলি করে হত্যা করেছিল পাকিস্তানি সেনারা। এই গণহত্যা হয়েছে শীর্ষ যুদ্ধাপরাধী গোলাম আযমের নির্দেশে। একাত্তরে পাকিস্তানি বাহিনীর বাঙালি হত্যার মূল হোতা ছিলেন তিনি।এক নম্বর অভিযোগে বলা হয়, ১৯৭১ সালের ৪ এপ্রিল গোলাম আযম, নুরুল আমীন, মৌলভী ফরিদ আহমেদ, খাজা খয়েরউদ্দিন, এ কে এম শফিকুল ইসলাম, মাওলানা নুরুজ্জামান, হামিদুল হক চৌধুরী, মোহসিনউদ্দিন আহ

In [13]:
X_train = pre_process(train_data['text'].values)
y_train = train_data['label'].values

In [14]:
labels = np.unique(y_train)

In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(X_train) #find out training accuracy

print('accuracy %s' % accuracy_score(y_pred, y_train))
print('accuracy normalized %s' % accuracy_score(y_pred, y_train, normalize=False))
print(classification_report(y_train, y_pred,target_names=labels))

accuracy 0.77
accuracy normalized 1386
              precision    recall  f1-score   support

          fe       0.47      0.98      0.63       300
          hm       0.90      0.89      0.89       300
          ij       0.98      0.69      0.81       300
          mk       0.88      0.72      0.79       300
          rg       0.98      0.54      0.70       300
          rn       0.94      0.80      0.86       300

    accuracy                           0.77      1800
   macro avg       0.86      0.77      0.78      1800
weighted avg       0.86      0.77      0.78      1800



In [16]:
nb.predict(X_train[:10])

array(['fe', 'fe', 'mk', 'rn', 'hm', 'hm', 'hm', 'fe', 'ij', 'rn'],
      dtype='<U2')

In [17]:
from sklearn.linear_model import SGDClassifier
svm = Pipeline([('vect', CountVectorizer()), #pipeline works one after another
               ('tfidf', TfidfTransformer()),
               ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
              ])
svm.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = svm.predict(X_train)

print('accuracy %s' % accuracy_score(y_pred, y_train))
print('accuracy normalized %s' % accuracy_score(y_pred, y_train, normalize=False))
print(classification_report(y_train, y_pred,target_names=labels))

accuracy 0.9672222222222222
accuracy normalized 1741
              precision    recall  f1-score   support

          fe       0.97      0.94      0.95       300
          hm       0.99      0.99      0.99       300
          ij       0.98      0.98      0.98       300
          mk       0.92      0.97      0.94       300
          rg       0.99      0.94      0.97       300
          rn       0.96      0.98      0.97       300

    accuracy                           0.97      1800
   macro avg       0.97      0.97      0.97      1800
weighted avg       0.97      0.97      0.97      1800



In [18]:

svm.predict(X_train[:10])

array(['fe', 'ij', 'mk', 'rn', 'hm', 'hm', 'ij', 'fe', 'ij', 'rn'],
      dtype='<U2')

In [19]:
X_test = pre_process(test_data['text'].values)
y_test = test_data['label'].values

In [20]:
# nb = Pipeline([('vect', CountVectorizer()),
#                ('tfidf', TfidfTransformer()),
#                ('clf', MultinomialNB()),
#               ])
# nb.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print('accuracy normalized %s' % accuracy_score(y_pred, y_test, normalize=False))
print(classification_report(y_test, y_pred,target_names=labels))

accuracy 0.6466666666666666
accuracy normalized 194
              precision    recall  f1-score   support

          fe       0.40      0.94      0.56        50
          hm       0.79      0.88      0.83        50
          ij       0.91      0.60      0.72        50
          mk       0.69      0.50      0.58        50
          rg       1.00      0.38      0.55        50
          rn       0.74      0.58      0.65        50

    accuracy                           0.65       300
   macro avg       0.76      0.65      0.65       300
weighted avg       0.76      0.65      0.65       300



In [21]:
from sklearn.metrics import classification_report
y_pred = svm.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print('accuracy normalized %s' % accuracy_score(y_pred, y_test, normalize=False))
print(classification_report(y_test, y_pred,target_names=labels))

accuracy 0.8333333333333334
accuracy normalized 250
              precision    recall  f1-score   support

          fe       0.84      0.74      0.79        50
          hm       0.92      0.94      0.93        50
          ij       0.84      0.94      0.89        50
          mk       0.69      0.90      0.78        50
          rg       0.97      0.72      0.83        50
          rn       0.81      0.76      0.78        50

    accuracy                           0.83       300
   macro avg       0.85      0.83      0.83       300
weighted avg       0.85      0.83      0.83       300



In [22]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
numpy.random.seed(7)

Using TensorFlow backend.


In [23]:
print(X_train[1])

সেতুং জন্মদিন তুং বারব ইতিহাস চাকা চৈনিক মানুষ মার্কস বিরুদ্ধ দাঁড়ি শ্রমিক কৃষক চিহ্নিত বিপ্লব মূল চালিকা শক্তি হিসেব তুল সশস্ত্র রেড আর্মি প্রচলন আরণ্যক গেরিলা যুদ্ধ সত্য বিশ্বম তরুণ হাঁট দেখানো নেপাল লাতিন আমেরিক বিপ্লবী তরুণ পার্বত্য আরণ্যক এলাকা নিঘূর্ম রাত চেয়ারম্যান নির্দেশ চিন হুনান নাম প্রদেশ হুনান প্রদেশ শাওশান গ্রাম ডিসেম্বর তুং জন্ম শাওশান গ্রাম পরিব স্বচ্ছল ভালো স্কুল বাবা কনফুসিয়াসপন্থি একনিষ্ট বৌদ্ধ কাজ ছেলেবেলা বিশ্বাসী হুনান প্রদেশ রাজধানীর নাম চাঙশা চাঙশা ভর্তি হুনান টির্চাস কলেজ পাশ্চাত্য দর্শন সম্বন্ধ বুদ্ধ কনফুসিয়াস কথা যাননি যাক সময় চিন চলছিল কিঙ রাজতন্ত্র দুঃশাসন বিরুদ্ধ জাতীয়তাবাদী তীব্র গনআন্দোলন সান ইয়াত জাতীয়তাবাদী নেতা রাজতন্ত্র ভেঙ্গ গঠন প্রজাতন্ত্র উদ্ধুদ্ধ যোগ প্রজাতন্ত্র সৈন্যবিভাগ রাজতন্ত্রবিরোধী আন্দোলন সান ইয়াত জয়ী জয়ী কউমিঙটাঙ জাতীয়তাবাদী গঠন প্রধান সান ইয়াত হুনান টির্চাস কলেজ পাস চাকরির খোঁজ বেজিং পৌঁছ জুটল বেজিং বিশ্ববিদ্যালয় লাইব্রেরি অখন্ড অবসর পাঠ লাগ সভ্যত বিস্ময়কর তত্ত্ব: মাকর্সবাদ বুঝ মায় বুদ্ধবাদ বাব কনফুসিয়বাদ কাজ জিনিষ এসব ব্যাক্তিদর্শন চ

In [24]:
#create the model
embedding_vector_length = 32 #here embedding is works like word2vec
model= Sequential()
model.add(Embedding(top_words+1, embedding_vector_length,input_length=max_review_length)) #here embedding starts from top word to 32,means there are 32 activation unit and each input length is 600 one hot encdoed vector
# so total layer is 5000*32 = 160032
model.add(LSTM(100)) # here add 100 layers of LSTM,each layer is loop with time slice,so this is deep LSTM,total param is 4(nm+n^2+n) where m=32,n=100
model.add(Dense(1,activation='sigmoid')) #here adding a one more layer which is sigmoid activation
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy']) #here binary cross entropy means log loss
print(model.summary())



NameError: name 'top_words' is not defined

In [ ]:
final_words.length()

In [ ]:
#applying CNN in this datasets
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))